# Notebook Title

## Setup Python and R environment
you can ignore this section

In [1]:
%load_ext rpy2.ipython
%load_ext autoreload
%autoreload 2

%matplotlib inline  
from matplotlib import rcParams
rcParams['figure.figsize'] = (16, 100)

import warnings
from rpy2.rinterface import RRuntimeWarning
warnings.filterwarnings("ignore") # Ignore all warnings
# warnings.filterwarnings("ignore", category=RRuntimeWarning) # Show some warnings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML

During startup - Warning messages:
1: Setting LC_COLLATE failed, using "C" 
2: Setting LC_TIME failed, using "C" 
3: Setting LC_MESSAGES failed, using "C" 
4: Setting LC_MONETARY failed, using "C" 


In [2]:
%%javascript
// Disable auto-scrolling
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [3]:
%%R

# My commonly used R imports

require('tidyverse')

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


Loading required package: tidyverse


## Load & Clean Data

👉 Load the data along with the census connectors below (the output of the `connect-to-census.ipynb` notebook) and do any cleanup you'd like to do.

In [5]:
import pandas as pd

In [6]:
df = pd.read_csv('active_tobacco_retailer_map_correct_cleaned_with_census.csv')
df

,LOCAL.HEALTH.UNIT,OPERATION.NAME,CITY,STATE,ZIP,MUNICIPALITY,COUNTY,VENDOR.TYPE,CREATION.DATE,LOCATION,LAT,LONG,ADDRESS,census_code,census_tract
0,NYC,2918 GAS CORP,BRONX,NY,10469,BRONX,NEW YORK CITY,OTHER,07/10/2023,"2918 BOSTON ROAD BRONX, NY 10469 (40.868058834...",40.868059,-73.858776,"2918 BOSTON ROAD BRONX, NY 10469",360050342002008,34200
1,NYC,SHISHA KING CORP,BROOKLYN,NY,11207,BROOKLYN,NEW YORK CITY,GROCERY SUPERMARKET,07/10/2023,"656 SUTTER AVENUE BROOKLYN, NY 11207 (40.66938...",40.669383,-73.894472,"656 SUTTER AVENUE BROOKLYN, NY 11207",360471156001001,115600
2,NYC,TUNI'S SERVICE CORP,BROOKLYN,NY,11215,BROOKLYN,NEW YORK CITY,CONVENIENCE STORE,07/10/2023,"306 18TH STREET BROOKLYN, NY 11215 (40.6617226...",40.661723,-73.989688,"306 18TH STREET BROOKLYN, NY 11215",360470143002001,14300
3,NYC,"VISHWA NEWS, INC.",BROOKLYN,NY,11201,BROOKLYN,NEW YORK CITY,STAND,07/10/2023,"17 BOND STREET BROOKLYN, NY 11201 (40.68917313...",40.689173,-73.983154,"17 BOND STREET BROOKLYN, NY 11201",360470037001002,3700
4,NYC,JAMAST HOLDING CORP,FLUSHING,NY,11358,QUEENS,NEW YORK CITY,CONVENIENCE STORE,07/10/2023,"161-17 29TH AVENUE FLUSHING, NY 11358 (40.7714...",40.771400,-73.802776,"161-17 29TH AVENUE FLUSHING, NY 11358",360811059004015,105900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5464,NYC,BULLPEN DELI INC,BRONX,NY,10451,BRONX,NEW YORK CITY,GROCERY SUPERMARKET,07/10/2023,"66 EAST 161ST STREET BRONX, NY 10451 (40.82737...",40.827372,-73.925434,"66 EAST 161ST STREET BRONX, NY 10451",360050063011003,6301
5465,NYC,HI MANGO 1055 INC,BROOKLYN,NY,11237,BROOKLYN,NEW YORK CITY,GROCERY SUPERMARKET,07/10/2023,"1055 FLUSHING AVENUE BROOKLYN, NY 11237 (40.70...",40.704307,-73.930115,"1055 FLUSHING AVENUE BROOKLYN, NY 11237",360470453002013,45300
5466,NYC,GREEN GARDEN 1 DELI CORP,BROOKLYN,NY,11229,BROOKLYN,NEW YORK CITY,CONVENIENCE STORE,07/10/2023,"2801 NOSTRAND AVENUE BROOKLYN, NY 11229 (40.61...",40.615752,-73.944714,"2801 NOSTRAND AVENUE BROOKLYN, NY 11229",360470644001002,64400
5467,NYC,DOPEBOYS EXOTIC INC,BROOKLYN,NY,11210,BROOKLYN,NEW YORK CITY,GROCERY SUPERMARKET,07/10/2023,"1437 FLATBUSH AVENUE BROOKLYN, NY 11210 (40.63...",40.635654,-73.950397,"1437 FLATBUSH AVENUE BROOKLYN, NY 11210",360470786011002,78601


In [7]:
# FILTER OUT MUNCIIPALITY = QUEENSBURY

df = df[df['MUNICIPALITY'] != 'QUEENSBURY']

df['MUNICIPALITY'].unique()

array(['BRONX', 'BROOKLYN', 'QUEENS', 'NEW YORK CITY', 'STATEN ISLAND'],
      dtype=object)

In [65]:
#remove last 3 digits from census_code
df['block_group'] = df['census_code'].apply(lambda x: str(x)[:-3])
df

,LOCAL.HEALTH.UNIT,OPERATION.NAME,CITY,STATE,ZIP,MUNICIPALITY,COUNTY,VENDOR.TYPE,CREATION.DATE,LOCATION,LAT,LONG,ADDRESS,census_code,census_tract,block_group
0,NYC,2918 GAS CORP,BRONX,NY,10469,BRONX,NEW YORK CITY,OTHER,07/10/2023,"2918 BOSTON ROAD BRONX, NY 10469 (40.868058834...",40.868059,-73.858776,"2918 BOSTON ROAD BRONX, NY 10469",360050342002008,34200,360050342002
1,NYC,SHISHA KING CORP,BROOKLYN,NY,11207,BROOKLYN,NEW YORK CITY,GROCERY SUPERMARKET,07/10/2023,"656 SUTTER AVENUE BROOKLYN, NY 11207 (40.66938...",40.669383,-73.894472,"656 SUTTER AVENUE BROOKLYN, NY 11207",360471156001001,115600,360471156001
2,NYC,TUNI'S SERVICE CORP,BROOKLYN,NY,11215,BROOKLYN,NEW YORK CITY,CONVENIENCE STORE,07/10/2023,"306 18TH STREET BROOKLYN, NY 11215 (40.6617226...",40.661723,-73.989688,"306 18TH STREET BROOKLYN, NY 11215",360470143002001,14300,360470143002
3,NYC,"VISHWA NEWS, INC.",BROOKLYN,NY,11201,BROOKLYN,NEW YORK CITY,STAND,07/10/2023,"17 BOND STREET BROOKLYN, NY 11201 (40.68917313...",40.689173,-73.983154,"17 BOND STREET BROOKLYN, NY 11201",360470037001002,3700,360470037001
4,NYC,JAMAST HOLDING CORP,FLUSHING,NY,11358,QUEENS,NEW YORK CITY,CONVENIENCE STORE,07/10/2023,"161-17 29TH AVENUE FLUSHING, NY 11358 (40.7714...",40.771400,-73.802776,"161-17 29TH AVENUE FLUSHING, NY 11358",360811059004015,105900,360811059004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5464,NYC,BULLPEN DELI INC,BRONX,NY,10451,BRONX,NEW YORK CITY,GROCERY SUPERMARKET,07/10/2023,"66 EAST 161ST STREET BRONX, NY 10451 (40.82737...",40.827372,-73.925434,"66 EAST 161ST STREET BRONX, NY 10451",360050063011003,6301,360050063011
5465,NYC,HI MANGO 1055 INC,BROOKLYN,NY,11237,BROOKLYN,NEW YORK CITY,GROCERY SUPERMARKET,07/10/2023,"1055 FLUSHING AVENUE BROOKLYN, NY 11237 (40.70...",40.704307,-73.930115,"1055 FLUSHING AVENUE BROOKLYN, NY 11237",360470453002013,45300,360470453002
5466,NYC,GREEN GARDEN 1 DELI CORP,BROOKLYN,NY,11229,BROOKLYN,NEW YORK CITY,CONVENIENCE STORE,07/10/2023,"2801 NOSTRAND AVENUE BROOKLYN, NY 11229 (40.61...",40.615752,-73.944714,"2801 NOSTRAND AVENUE BROOKLYN, NY 11229",360470644001002,64400,360470644001
5467,NYC,DOPEBOYS EXOTIC INC,BROOKLYN,NY,11210,BROOKLYN,NEW YORK CITY,GROCERY SUPERMARKET,07/10/2023,"1437 FLATBUSH AVENUE BROOKLYN, NY 11210 (40.63...",40.635654,-73.950397,"1437 FLATBUSH AVENUE BROOKLYN, NY 11210",360470786011002,78601,360470786011


In [66]:
#export as csv python
df.to_csv('active_tobacco_retailers_final.csv', index=False)
print('done')

done


## 👉 Grab Census Data

1. loading the Census API key

In [8]:
import dotenv

# Load the environment variables
# (loads CENSUS_API_KEY from .env)
dotenv.load_dotenv()


True

In [9]:
%%R 

require('tidycensus')

# because it an environment variable, we don't have to 
# explicitly pass this string to R, it is readable here
# in this R cell.
census_api_key(Sys.getenv("CENSUS_API_KEY"))

Loading required package: tidycensus
To install your API key for use in future sessions, run this function with `install = TRUE`.


2. Decide which Census variables you want

    Use <https://censusreporter.org/> to figure out which tables you want. (if censusreporter is down, check out the code in the cell below)

    -   Scroll to the bottom of the page to see the tables.
    -   If you already know the table ID, stick that in the "Explore" section to learn more about that table.

    By default this code loads (B01003_001) which we found in censusreporter here: https://censusreporter.org/tables/B01003/

    - find some other variables that you're also interested in
    - don't forget to pick a geography like "tract", "county" or "block group". here is the list of [all geographies](https://walker-data.com/tidycensus/articles/basic-usage.html#geography-in-tidycensus
    ).


In [11]:
# %%R 

# # Finding the Census Varaibles for the ACS 5 year survey
# # Generall you'd do this in CensusReporter, but since it's down sometimes, here it is using tidycensus's load_variables function

# # get every single variable in the ACS5
# all_census_vars <- load_variables(2021, "acs5", cache = TRUE) 

# filtered_census_vars <- all_census_vars %>% 
#     filter(grepl("median income", label, ignore.case = TRUE))   # filter to those containing "median income"
    
# # write to CSV so we can view it in python
# filtered_census_vars %>% 
#     write_csv("filtered_census_vars.csv")

# # show the first few rows
# filtered_census_vars %>%
#     select(-geography) %>% # remove the geography column
#     print(n = 20) # print the first 20 rows

In [10]:
%%R 
# the variable B01003_001E was selectd from the census table 
# for population, which we found in censusreporter here:
# https://censusreporter.org/tables/B01003/

# in the table below, pick the geography, the variables, and the survey you want to pull from
# see the possible values here https://walker-data.com/tidycensus/articles/basic-usage.html

# Get variable from ACS
nyc_census_data <- get_acs(geography = "block group", 
                      state='NY',
                      county = c("New York", "Kings", "Queens", "Bronx", "Richmond"),
                      variables = c(
                        # population="B01003_001",
                        med_inc="B19013_001"
                      ), 
                      year = 2021,
                      survey="acs5",
                      geometry=T)

nyc_census_data

  |======================================================================| 100%
Simple feature collection with 6807 features and 5 fields (with 49 geometries empty)
Geometry type: MULTIPOLYGON
Dimension:     XY
Bounding box:  xmin: -74.25609 ymin: 40.4961 xmax: -73.70036 ymax: 40.91771
Geodetic CRS:  NAD83
First 10 features:
          GEOID                                                          NAME
1  360810934012   Block Group 2, Census Tract 934.01, Queens County, New York
2  360610135020 Block Group 0, Census Tract 135.02, New York County, New York
3  360050516012    Block Group 2, Census Tract 516.01, Bronx County, New York
4  360610150011 Block Group 1, Census Tract 150.01, New York County, New York
5  360470333001       Block Group 1, Census Tract 333, Kings County, New York
6  360610089003     Block Group 3, Census Tract 89, New York County, New York
7  360810004002        Block Group 2, Census Tract 4, Queens County, New York
8  360850125002    Block Group 2, Census Tract 12

Getting data from the 2017-2021 5-year ACS
Using FIPS code '36' for state 'NY'
Using FIPS code '061' for 'New York County'
Using FIPS code '047' for 'Kings County'
Using FIPS code '081' for 'Queens County'
Using FIPS code '005' for 'Bronx County'
Using FIPS code '085' for 'Richmond County'


## 👉 Merge it with your data

hint...`tidycensus` provides you data in long format you may need to pivot the census data from long to wide format before merging it with your data

In [11]:
%%R

nyc_census_data_wider <- nyc_census_data %>% 
  # pivot from wide to long
  pivot_wider(
    names_from=variable, 
    values_from = c(estimate, moe),
    names_glue = "{variable}_{.value}"
  )

nyc_census_data_wider

Simple feature collection with 6807 features and 4 fields (with 49 geometries empty)
Geometry type: MULTIPOLYGON


Dimension:     XY
Bounding box:  xmin: -74.25609 ymin: 40.4961 xmax: -73.70036 ymax: 40.91771
Geodetic CRS:  NAD83
# A tibble: 6,807 × 5
   GEOID        NAME                       geometry med_inc_estimate med_inc_moe
   <chr>        <chr>            <MULTIPOLYGON [°]>            <dbl>       <dbl>
 1 360810934012 Block Gr… (((-73.844 40.57951, -73…            82000       22962
 2 360610135020 Block Gr… (((-74.00162 40.77455, -…               NA          NA
 3 360050516012 Block Gr… (((-73.79188 40.85522, -…           118514       31053
 4 360610150011 Block Gr… (((-73.95881 40.78086, -…           205455      108346
 5 360470333001 Block Gr… (((-73.94244 40.66673, -…            85313       26481
 6 360610089003 Block Gr… (((-74.00553 40.74582, -…               NA          NA
 7 360810004002 Block Gr… (((-73.86806 40.69118, -…            78500       28660
 8 360850125002 Block Gr… (((-74.12206 40.62836, -…            81875       23435
 9 360470944012 Block Gr… (((-73.92679 40.63611, -…  

In [60]:
nyc_census_data_wider[nyc_census_data_wider['GEOID'] == '360050342002']

,GEOID,NAME,med_inc_estimate,med_inc_moe
816,360050342002,"Block Group 2, Census Tract 342, Bronx County,...",108684.0,41287.0


In [5]:
%%R -o df_tobacco

#import active tobacco retailer data
df_tobacco <- read.csv('active_tobacco_retailers_final.csv')

In [69]:
%%R

#column headers for wider census data
colnames(nyc_census_data_wider)

[1]

 "GEOID"            "NAME"             "geometry"         "med_inc_estimate"
[5] "med_inc_moe"     


In [68]:
%%R

#census data wide name head
head(nyc_census_data_wider$NAME)

[1] "Block Group 2, Census Tract 934.01, Queens County, New York"  
[2] "Block Group 0, Census Tract 135.02, New York County, New York"
[3] "Block Group 2, Census Tract 516.01, Bronx County, New York"   
[4] "Block Group 1, Census Tract 150.01, New York County, New York"
[5] "Block Group 1, Census Tract 333, Kings County, New York"      
[6] "Block Group 3, Census Tract 89, New York County, New York"    


In [70]:
%%R

#df_tobacco numnber of unique census_code 
length(unique(df_tobacco$block_group))

[1] 2914


In [6]:
%%R -o df_tobacco_grouped

# group tobacco data by census_code and count the number of tobacco retailers in each census_code
df_tobacco_grouped <- df_tobacco %>% 
  group_by(block_group) %>% 
  summarise(n=n())

In [87]:
%%R

head(df_tobacco_grouped)

# A tibble: 6 × 2
   block_group     n
         <dbl> <int>
1 360050002001     1
2 360050002002     1
3 360050004001     3
4 360050004002     1
5 360050016001     1
6 360050016002     1


In [84]:
%%R

#number of rows in df_tobacco_grouped
nrow(df_tobacco_grouped)

[1] 2914


In [92]:
%%R

nrow(nyc_census_data_wider)

[1] 6807


In [85]:
%%R

#df_tobacco_grouped unique in n

unique(df_tobacco_grouped$n)

 [1]  1  3  2  4  7  6  5  8  9 10


In [88]:
%%R

# cast df_tobacco_grouped$census_code as character
df_tobacco_grouped$block_group <- as.character(df_tobacco_grouped$block_group)

df_tobacco_grouped$block_group

   [1]

 "360050002001" "360050002002" "360050004001" "360050004002"
   [5] "360050016001" "360050016002" "360050019011" "360050019021"
   [9] "360050020021" "360050020023" "360050023001" "360050025001"
  [13] "360050025003" "360050025004" "360050027012" "360050027021"
  [17] "360050027022" "360050028001" "360050031001" "360050033002"
  [21] "360050033003" "360050035001" "360050035003" "360050039001"
  [25] "360050039002" "360050039003" "360050039005" "360050041001"
  [29] "360050041003" "360050041004" "360050042001" "360050042005"
  [33] "360050043001" "360050043002" "360050043003" "360050043004"
  [37] "360050044002" "360050044003" "360050046001" "360050048001"
  [41] "360050048002" "360050050011" "360050050013" "360050050021"
  [45] "360050050022" "360050050023" "360050051001" "360050051002"
  [49] "360050052001" "360050053001" "360050054002" "360050054003"
  [53] "360050054004" "360050056003" "360050059021" "360050062004"
  [57] "360050062005" "360050063011" "360050063014" "360050063015"
 

In [25]:
%%R

# how many rows in df_tobacco_grouped don't overlap with nyc_census_data_wider using GEOID and block_group as the reference
sum(!df_tobacco_grouped$block_group %in% nyc_census_data_wider$GEOID)

[1] 2


In [26]:
%%R

# can you show the rows where the block_group doesn't overlap with nyc_census_data_wider
df_tobacco_grouped[!df_tobacco_grouped$block_group %in% nyc_census_data_wider$GEOID,]

# A tibble: 2 × 2
   block_group     n
         <dbl> <int>
1 360593022001     1
2 360594062022     1


In [27]:
%%R

# drop the above two rows from df_tobacco_grouped
df_tobacco_grouped <- df_tobacco_grouped[df_tobacco_grouped$block_group %in% nyc_census_data_wider$GEOID,]

In [28]:
%%R

# how many rows in df_tobacco_grouped don't overlap with nyc_census_data_wider using GEOID and block_group as the reference
sum(!df_tobacco_grouped$block_group %in% nyc_census_data_wider$GEOID)

[1] 0


In [29]:
%%R

#merge census data with tobacco data on GEOID and block_group
merged_data <- merge(nyc_census_data_wider, df_tobacco_grouped, by.x = 'GEOID', by.y = 'block_group', all.x = TRUE)

In [15]:
%%R

head(merged_data)

Simple feature collection with 6 features and 5 fields
Geometry type: MULTIPOLYGON
Dimension:     XY
Bounding box:  xmin: -73.89987 ymin: 40.78536 xmax: -73.85442 ymax: 40.81527
Geodetic CRS:  NAD83
         GEOID                                                  NAME
1 360050001000 Block Group 0, Census Tract 1, Bronx County, New York
2 360050001001 Block Group 1, Census Tract 1, Bronx County, New York
3 360050002000 Block Group 0, Census Tract 2, Bronx County, New York
4 360050002001 Block Group 1, Census Tract 2, Bronx County, New York
5 360050002002 Block Group 2, Census Tract 2, Bronx County, New York
6 360050002003 Block Group 3, Census Tract 2, Bronx County, New York
  med_inc_estimate med_inc_moe  n                       geometry
1               NA          NA NA MULTIPOLYGON (((-73.8925 40...
2               NA          NA NA MULTIPOLYGON (((-73.89277 4...
3               NA          NA NA MULTIPOLYGON (((-73.86648 4...
4               NA          NA  1 MULTIPOLYGON (((-73.8633

In [23]:
%%R

library(sf)

Linking to GEOS 3.12.1, GDAL 3.8.3, PROJ 9.3.1; sf_use_s2() is TRUE


In [30]:
%%R -o merged_data_python

#remove geometry column
merged_data_python <- merged_data %>% 
    #drop geometries
    st_drop_geometry()

In [91]:
%%R

head(merged_data)

Simple feature collection with 6 features and 5 fields
Geometry type: MULTIPOLYGON
Dimension:     XY
Bounding box:  xmin: -73.89987 ymin: 40.78536 xmax: -73.85442 ymax: 40.81527
Geodetic CRS:  NAD83
         GEOID                                                  NAME
1 360050001000 Block Group 0, Census Tract 1, Bronx County, New York
2 360050001001 Block Group 1, Census Tract 1, Bronx County, New York
3 360050002000 Block Group 0, Census Tract 2, Bronx County, New York
4 360050002001 Block Group 1, Census Tract 2, Bronx County, New York
5 360050002002 Block Group 2, Census Tract 2, Bronx County, New York
6 360050002003 Block Group 3, Census Tract 2, Bronx County, New York
  med_inc_estimate med_inc_moe  n                       geometry
1               NA          NA NA MULTIPOLYGON (((-73.8925 40...
2               NA          NA NA MULTIPOLYGON (((-73.89277 4...
3               NA          NA NA MULTIPOLYGON (((-73.86648 4...
4               NA          NA  1 MULTIPOLYGON (((-73.8633

In [94]:
%%R

#number of rows in merged_data
nrow(merged_data)

[1] 6807


In [96]:
%%R

#renam n to num_tobacco_retailers
names(merged_data)[names(merged_data) == 'n'] <- 'num_tobacco_retailers'

In [97]:
%%R

#export merged data as csv
write.csv(merged_data, 'merged_data.csv')